In [1]:
# -*- coding: utf-8 -*-
"""
@author:

Python Parser for the JSON files
"""

import pandas as pd
import numpy as np
import json
# import ijson

In [2]:
def load_jsonfile(jsonfile):
    #
    data = []
    with open(jsonfile, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data


In [3]:
jsonfile = "E:/_temp_/data check/data.jsonl"

data = load_jsonfile(jsonfile)


In [4]:
print("There are {} journeys in the file {}".format(len(data), jsonfile))
jourey_index = 0  # take first journey to check

x = pd.DataFrame(data[jourey_index]['Route']).sort_values(['Timestamp'], ascending=True)
print(x.shape)
x.head()

There are 2000 journeys in the file E:/_temp_/data.jsonl
(414, 8)


,Accuracy,Bearing,Height,Latitude,Longitude,Offset,Speed,Timestamp
0,25.0,192.33626,137.929953,53.222135,-6.655491,4464,2.481998,1499351854000
1,20.0,232.44154,145.769546,53.222113,-6.655550,5463,3.579896,1499351855000
2,20.0,269.07810,150.182644,53.222132,-6.655593,6461,4.783836,1499351856000
3,20.0,288.23306,149.075309,53.222164,-6.655655,7464,5.375758,1499351857000
4,25.0,295.18048,153.039079,53.222200,-6.655714,8461,5.017635,1499351858000


In [5]:
print("Sample Journey ID is {}".format(data[jourey_index]['Journey ID']))
triplength_ = round((max(x['Timestamp'])-min(x['Timestamp']))/3600000., 3)
print("Its total trip length is {} hours ({} millseconds)".format(triplength_, (max(x['Timestamp'])-min(x['Timestamp']))))
mean_speed_ = np.round(np.mean(x['Speed']), 3)
print("Its average speed is {} m/s".format(mean_speed_))

Sample Journey ID is 3ce0d93b-5aa2-481b-852d-c80adfd4685b.6jul2017143655
Its total trip length is 0.119 hours (429000 millseconds)
Its average speed is 3.827 m/s


##  Insert Accleromter Data into Route Data

In [6]:
x2 = pd.DataFrame(data[0]['Accelerometer']).sort_values(['Offset'], ascending=True)
x2.head()

,Offset,x,y,z
0,84,3.106477,1.423353,9.020155
1,193,3.092112,1.440112,9.021352
2,302,3.100491,1.449689,9.027337
3,403,3.096900,1.449689,9.053674
4,513,3.110068,1.446098,9.062054


In [7]:
# Now attatch offset data back to route data

bins = list(x['Offset'].values)
group_names = list(range(1,len(bins)))
x2['offsetbin'] = pd.cut(x2['Offset'], bins, labels=group_names)
x2 = x2.loc[x2.offsetbin.notnull(), :]
x2.rename(columns={"x":"acc_x", "y":"acc_y", "z":"acc_z", "Offset":"Offset_temp"}, inplace=True)
x2.head()

,Offset_temp,acc_x,acc_y,acc_z,offsetbin
42,4543,4.097676,0.752976,9.647436,1
43,4643,4.747702,0.290895,9.817424,1
44,4753,4.509479,-0.293290,9.354146,1
45,4862,3.881001,-0.913388,8.631097,1
46,4963,3.721786,-0.968455,8.519767,1


In [8]:
x["offsetbin"] = range(1, x.shape[0]+1)
x = pd.merge(x, x2, on='offsetbin', how='left')
print(x.shape)
x = pd.DataFrame(x.sort_values(['Timestamp', 'Offset'], ascending=True))
del x2

(4074, 13)


In [9]:
x["index1"] = x.index
x_ = x.groupby(['offsetbin'])['index1'].agg([np.min])

In [10]:
x_["offsetbin"] = x_.index
x_.columns.values[0] = "index1"
x_.reset_index(drop=True, inplace =True)

In [11]:
x2 = pd.merge(x, x_, on=['offsetbin', 'index1'], how='inner')
del x_
x2.head()

,Accuracy,Bearing,Height,Latitude,Longitude,Offset,Speed,Timestamp,offsetbin,Offset_temp,acc_x,acc_y,acc_z,index1
0,25.0,192.33626,137.929953,53.222135,-6.655491,4464,2.481998,1499351854000,1,4543.0,4.097676,0.752976,9.647436,0
1,20.0,232.44154,145.769546,53.222113,-6.655550,5463,3.579896,1499351855000,2,5483.0,3.938462,-0.257377,9.312247,9
2,20.0,269.07810,150.182644,53.222132,-6.655593,6461,4.783836,1499351856000,3,6553.0,3.023877,-1.413776,8.402451,19
3,20.0,288.23306,149.075309,53.222164,-6.655655,7464,5.375758,1499351857000,4,7513.0,3.542221,0.562637,8.998607,28
4,25.0,295.18048,153.039079,53.222200,-6.655714,8461,5.017635,1499351858000,5,8465.0,2.930503,2.505532,9.236830,37


In [12]:
x2.loc[:, ['Offset_temp','acc_x', 'acc_y', 'acc_z']] = np.nan
x2.drop(['index1'], axis=1, inplace=True)
x.drop(['index1'], axis=1, inplace=True)
x = x.append(x2)

In [13]:
x.loc[x.Offset_temp.isnull(), 'Offset_temp'] = -1
x=x.sort_values(['Timestamp', 'Offset_temp'], ascending=[True, True])
x.loc[x.Offset_temp > 0, ["Accuracy", "Bearing", "Height", "Latitude", "Longitude", "Offset", "Speed", "Timestamp"]] = np.nan

In [14]:
x.drop(['offsetbin'], axis=1, inplace=True)
x.loc[x.Offset.isnull(), 'Offset'] = x.loc[x.Offset.isnull(), 'Offset_temp']
x.drop(['Offset_temp'], axis=1, inplace=True)
print(x.shape)
x.head(15)

(4488, 11)


,Accuracy,Bearing,Height,Latitude,Longitude,Offset,Speed,Timestamp,acc_x,acc_y,acc_z
0,25.0,192.33626,137.929953,53.222135,-6.655491,4464.0,2.481998,1.499352e+12,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,4543.0,NaN,NaN,4.097676,0.752976,9.647436
1,NaN,NaN,NaN,NaN,NaN,4643.0,NaN,NaN,4.747702,0.290895,9.817424
2,NaN,NaN,NaN,NaN,NaN,4753.0,NaN,NaN,4.509479,-0.293290,9.354146
3,NaN,NaN,NaN,NaN,NaN,4862.0,NaN,NaN,3.881001,-0.913388,8.631097
4,NaN,NaN,NaN,NaN,NaN,4963.0,NaN,NaN,3.721786,-0.968455,8.519767
5,NaN,NaN,NaN,NaN,NaN,5062.0,NaN,NaN,4.078523,-0.896629,8.503007
6,NaN,NaN,NaN,NaN,NaN,5162.0,NaN,NaN,4.337096,-0.409408,9.378088
7,NaN,NaN,NaN,NaN,NaN,5263.0,NaN,NaN,4.320337,-0.137667,8.954314
8,NaN,NaN,NaN,NaN,NaN,5373.0,NaN,NaN,4.161122,-0.918176,8.617929
